In [3]:
import requests
import os
from bs4 import BeautifulSoup
from openai import OpenAI
from dotenv import load_dotenv

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)



In [4]:
system_prompt = "You are an assistant who analyzes content of a website given the content and will summarize it. Ignore navigation related information."


def get_user_prompt(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
        please provide a short summary of this website in markdown. \
        If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt




In [5]:
def summarize(url):
    web = Website(url)
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": get_user_prompt(web)}
        ]
    openai = OpenAI()
    response = openai.chat.completions.create(model = "gpt-4o-mini", messages = messages)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages
    )
    print(response.choices[0].message.content)
    

In [6]:
load_dotenv()

openapi_key = os.getenv('OPENAI_API_KEY')

summarize("https://cnn.com")

# Summary of Edward Donner's Website

The website is maintained by Ed Donner, a co-founder and CTO of **Nebula.io**, a company focused on harnessing AI to enhance talent discovery and management. Ed has a background in coding, experimentation with LLMs (Large Language Models), and a passion for DJing and amateur electronic music.

## Key Features:
- **Outsmart**: An innovative arena where LLMs compete in diplomacy and strategy.
- **About Ed**: Ed describes his interests, his journey in tech, and his previous role as the founder and CEO of AI startup **untapt**, which was acquired in 2021.
- **Connect**: Visitors are encouraged to connect with Ed, and he offers a newsletter subscription for updates.

## News and Announcements:
- **December 21, 2024**: Welcome to SuperDataScientists!
- **November 13, 2024**: Mastering AI and LLM Engineering – Resources.
- **October 16, 2024**: From Software Engineer to AI Data Scientist – Resources.
- **August 6, 2024**: Introduction to the Outsmart LLM 